<a href="https://colab.research.google.com/github/madalapranavsai/AI_agent/blob/main/pranav_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [44]:
!pip install -q google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

# STEP 1: Imports and secure API key input

In [45]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [46]:

os.environ['GOOGLE_API_KEY']=getpass.getpass("Enter Gemini API Key:")

Enter Gemini API Key:··········


# STEP 2: Initialize Gemini 1.5 Flash

In [58]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest",temperature=0.2)

# STEP 3: Node to ask user for symptom

In [48]:
 def get_symptom_node(state:dict)->dict:
  symptom=input("welcome enter your symptom")
  state["symptom"]=symptom
  return state

# STEP 4: Node to classify the symptom

In [54]:
from sre_constants import CATEGORY
def classify_symptom_node(state:dict)->dict:
  prompt= (
      "you are a helpful medical assistant , classisfy he symptoms in one the categories \n"
      "General\n-Emergency\n-mentalhealth\n"
      f"symptom : {state['symptom']}\n"
      "respond only with one word : General , Emergency Or mentalhealth "
      "#example : input : I Have Fever , Output : General "
  )
  response=llm.invoke([HumanMessage(content=prompt)])
  CATEGORY =response.content.strip()
  print(f"llm classifies  the symptom as {CATEGORY}")
  state["CATEGORY"]=CATEGORY
  return state

# STEP 5: Router logic to route to the correct node


In [50]:
def symptom_router(state:dict)->dict:
  cat= state["CATEGORY"].lower()
  if "general" in cat :
      return "general"

  elif "emergency" in cat:
      return "emergency"
  elif "mentalhealth" in cat:
      return "mentalhealth"
  else:
      return "general"

# STEP 6: Category-specific response nodes


In [55]:
def general_node(state:dict)->dict:
  state["answer"]=f"'{state['symptom']} ': directing you to consult a general doctor"
  return state
def emergency_node(state:dict)->dict:
  state["answer"]=f"'{state['symptom']} ': directing you to consult a emergency doctor"
  return state
def mentalhealth_node(state:dict)->dict:
  state["answer"]=f"'{state['symptom']} ': directing you to consult a mental doctor"
  return state

# STEP 7: Build LangGraph


In [56]:
bulider=StateGraph(dict)
bulider.set_entry_point("get_symptom")
bulider.add_node("get_symptom",get_symptom_node)
bulider.add_node("classify",classify_symptom_node)
bulider.add_node("general",general_node)
bulider.add_node("emergency",emergency_node)
bulider.add_node("mentalhealth",mentalhealth_node)
bulider.add_edge("get_symptom","classify")
bulider.add_conditional_edges("classify",symptom_router,{
    "general":"general",
    "emergency":"emergency",
    "mentalhealth":"mentalhealth"
})
bulider.add_edge("general",END)
bulider.add_edge("emergency",END)
bulider.add_edge("mentalhealth",END)

# STEP 8: Compile and invoke the graph


In [60]:
graph= bulider.compile()
final_state=graph.invoke({})
print(final_state["answer"])

welcome enter your symptomhigh fever with cough and blood vomit
llm classifies  the symptom as Emergency
'high fever with cough and blood vomit ': directing you to consult a emergency doctor
